# Pytorch图像分类模型转ONNX-水果30类

同济子豪兄 https://space.bilibili.com/1900783

代码运行云GPU平台：公众号 人工智能小技巧 回复 gpu

2022-8-22 2023-5-8

## 导入工具包

In [1]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 下载自己训练得到的30类水果图像分类Pytorch模型权重文件

In [2]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth -P checkpoint


--2023-05-08 09:59:48--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 44854477 (43M) [binary/octet-stream]
正在保存至: “checkpoint/fruit30_pytorch_20220814.pth”

fruit30_pytorch_202 100%[===================>]  42.78M  23.1MB/s    用时 1.9s    

2023-05-08 09:59:51 (23.1 MB/s) - 已保存 “checkpoint/fruit30_pytorch_20220814.pth” [44854477/44854477])



## 导入训练好的模型

In [3]:
model = torch.load('checkpoint/fruit30_pytorch_20220814.pth')
model = model.eval().to(device)

## 构造一个输入图像Tensor

In [4]:
x = torch.randn(1, 3, 256, 256).to(device)

## 输入Pytorch模型推理预测，获得1000个类别的预测结果

In [5]:
output = model(x)

In [6]:
output.shape

torch.Size([1, 30])

## Pytorch模型转ONNX模型

In [7]:
x = torch.randn(1, 3, 256, 256).to(device)

with torch.no_grad():
    torch.onnx.export(
        model,                   # 要转换的模型
        x,                       # 模型的任意一组输入
        'resnet18_fruit30.onnx', # 导出的 ONNX 文件名
        opset_version=11,        # ONNX 算子集版本
        input_names=['input'],   # 输入 Tensor 的名称（自己起名字）
        output_names=['output']  # 输出 Tensor 的名称（自己起名字）
    ) 

## 验证onnx模型导出成功

In [8]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('resnet18_fruit30.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [9]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 30x512]
  %fc.bias[FLOAT, 30]
  %193[FLOAT, 64x3x7x7]
  %194[FLOAT, 64]
  %196[FLOAT, 64x64x3x3]
  %197[FLOAT, 64]
  %199[FLOAT, 64x64x3x3]
  %200[FLOAT, 64]
  %202[FLOAT, 64x64x3x3]
  %203[FLOAT, 64]
  %205[FLOAT, 64x64x3x3]
  %206[FLOAT, 64]
  %208[FLOAT, 128x64x3x3]
  %209[FLOAT, 128]
  %211[FLOAT, 128x128x3x3]
  %212[FLOAT, 128]
  %214[FLOAT, 128x64x1x1]
  %215[FLOAT, 128]
  %217[FLOAT, 128x128x3x3]
  %218[FLOAT, 128]
  %220[FLOAT, 128x128x3x3]
  %221[FLOAT, 128]
  %223[FLOAT, 256x128x3x3]
  %224[FLOAT, 256]
  %226[FLOAT, 256x256x3x3]
  %227[FLOAT, 256]
  %229[FLOAT, 256x128x1x1]
  %230[FLOAT, 256]
  %232[FLOAT, 256x256x3x3]
  %233[FLOAT, 256]
  %235[FLOAT, 256x256x3x3]
  %236[FLOAT, 256]
  %238[FLOAT, 512x256x3x3]
  %239[FLOAT, 512]
  %241[FLOAT, 512x512x3x3]
  %242[FLOAT, 512]
  %244[FLOAT, 512x256x1x1]
  %245[FLOAT, 512]
  %247[FLOAT, 512x512x3x3]
  %248[FLOAT, 512]
  %250[FLOAT, 512x512x3

## 使用Netron可视化模型结构

Netron：https://netron.app

视频教程：https://www.bilibili.com/video/BV1TV4y1P7AP